# Current members of the U.S. Congress

In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import requests
import matplotlib.pyplot as plt
import numpy as np
import json
import os
import seaborn as sns
import altair as alt

In [3]:
key = "sg5Q11oWW9pqLtmn8gVS0nOS179RousHDpeTgZkA"

In [4]:
formaturl = (
    lambda x: "https://api.propublica.org/congress/v1/"
    + f"{x[0]}/"
    + f"{x[1]}"
    + "/members.json"
)

In [5]:
metadata = []
for s in range(117, 118):
    for c in ["house", "senate"]:
        metadata.append(dict(chamber=c, session=s, url=formaturl((s, c))))

In [6]:
jsons = []
for m in metadata:
    r = requests.get(m["url"], headers={"X-API-KEY": key})
    d = r.json()
    m["datadict"] = d
    jsons.append(m)

In [7]:
def processdict(d):
    data = pd.json_normalize(d["datadict"]["results"], record_path=["members"])
    data["chamber"] = d["chamber"]
    data["session"] = d["session"]
    return data

In [8]:
ap_states = {
    "AL": "Ala.",
    "AK": "Alaska",
    "AZ": "Ariz.",
    "AR": "Ark.",
    "CA": "Calif.",
    "CO": "Colo.",
    "CT": "Conn.",
    "DE": "Del.",
    "FL": "Fla.",
    "GA": "Ga.",
    "HI": "Hawaii",
    "IA": "Iowa",
    "ID": "Idaho",
    "IL": "Ill.",
    "IN": "Ind.",
    "KS": "Kan.",
    "KY": "Ky.",
    "LA": "La.",
    "MD": "Md.",
    "MA": "Mass.",
    "ME": "Maine",
    "MI": "Mich.",
    "MN": "Minn.",
    "MS": "Miss.",
    "MO": "Mo.",
    "MT": "Mont.",
    "NE": "Neb.",
    "NV": "Nev.",
    "NH": "N.H.",
    "NJ": "N.J.",
    "NM": "N.M.",
    "NY": "N.Y.",
    "NC": "N.C.",
    "ND": "N.D.",
    "OH": "Ohio",
    "OK": "Okla.",
    "OR": "Ore.",
    "PA": "Pa.",
    "RI": "R.I.",
    "SC": "S.C.",
    "SD": "S.D.",
    "TX": "Texas",
    "TN": "Tenn.",
    "VT": "Vt.",
    "VA": "Va.",
    "WA": "Wash.",
    "WV": "W.Va.",
    "WI": "Wis.",
    "WY": "Wyo",
}

In [9]:
src = pd.concat([processdict(j) for j in jsons], sort=False)

In [10]:
src["apstate"] = src["state"].map(ap_states)

In [11]:
src["full_name"] = src["first_name"] + " " + src["last_name"]

In [20]:
src.head()

,id,title,short_title,api_uri,first_name,middle_name,last_name,suffix,date_of_birth,gender,...,missed_votes_pct,votes_with_party_pct,votes_against_party_pct,chamber,session,senate_class,state_rank,lis_id,apstate,full_name
0,A000370,Representative,Rep.,https://api.propublica.org/congress/v1/members...,Alma,None,Adams,None,1946-05-27,F,...,0.67,98.65,0.90,house,117,NaN,NaN,NaN,N.C.,Alma Adams
1,A000055,Representative,Rep.,https://api.propublica.org/congress/v1/members...,Robert,B.,Aderholt,None,1965-07-22,M,...,1.78,97.02,2.52,house,117,NaN,NaN,NaN,Ala.,Robert Aderholt
2,A000371,Representative,Rep.,https://api.propublica.org/congress/v1/members...,Pete,None,Aguilar,None,1979-06-19,M,...,0.22,98.43,1.12,house,117,NaN,NaN,NaN,Calif.,Pete Aguilar
3,A000372,Representative,Rep.,https://api.propublica.org/congress/v1/members...,Rick,None,Allen,None,1951-11-07,M,...,2.00,93.12,6.42,house,117,NaN,NaN,NaN,Ga.,Rick Allen
4,A000376,Representative,Rep.,https://api.propublica.org/congress/v1/members...,Colin,None,Allred,None,1983-04-15,M,...,0.00,97.54,2.01,house,117,NaN,NaN,NaN,Texas,Colin Allred


In [12]:
df = src[
    [
        "id",
        "seniority",
        "full_name",
        "apstate",
        "first_name",
        "last_name",
        "chamber",
        "session",
        "state",
        "party",
        "dw_nominate",
        "votes_with_party_pct",
    ]
]

In [13]:
len(df)

548

In [19]:
df.head()

,id,seniority,full_name,apstate,first_name,last_name,chamber,session,state,party,dw_nominate,votes_with_party_pct
0,A000370,10,Alma Adams,N.C.,Alma,Adams,house,117,NC,D,-0.465,98.65
1,A000055,26,Robert Aderholt,Ala.,Robert,Aderholt,house,117,AL,R,0.376,97.02
2,A000371,8,Pete Aguilar,Calif.,Pete,Aguilar,house,117,CA,D,-0.294,98.43
3,A000372,8,Rick Allen,Ga.,Rick,Allen,house,117,GA,R,0.696,93.12
4,A000376,4,Colin Allred,Texas,Colin,Allred,house,117,TX,D,NaN,97.54


---

### Export

In [15]:
df.to_csv("output/members_of_congress_117.csv")